그라디오 챗봇와 랭체인 LLM 연동하기

In [ ]:
# 그라디오 라이브러리를 설치합니다.
#poetry add gradio 
# 랭체인 라이브러리를 설치합니다.
#poetry add langchain 
# 랭체인 OpenAI 연동 라이브러리를 설치합니다.
#poetry add langchain-openai

#### LangChain의 ChatOpenAI가 환경 변수를 자동으로 로드하는 방식
* load_dotenv()를 실행하면 .env 파일에 있는 환경 변수가 시스템 환경 변수로 등록됨
* LangChain의 ChatOpenAI는 api_key=None일 때 os.getenv("OPENAI_API_KEY")를 자동으로 불러옴
* 따라서 api_key를 명시적으로 설정하지 않아도 API 키가 자동으로 적용됨

In [ ]:
from dotenv import load_dotenv
import os

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:4])

In [ ]:
import langchain

print(langchain.__version__)

In [ ]:
from langchain_openai import ChatOpenAI

# LLM 인스턴스 생성
#llm = ChatOpenAI(model_name="gpt-3.5-turbo")
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

response = llm.invoke([{"role": "user", "content": "파이썬이란?"}])
print(response)
print(response.content)

#### API 키를 명시적으로 설정하는 방법

In [ ]:
from langchain_openai import ChatOpenAI

# 명시적으로 API 키 설정
#llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)  
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

response = llm.invoke([{"role": "user", "content": "파이썬이란?"}])
print(response)

In [ ]:
# 그라디오 라이브러리를 불러옵니다.
import gradio as gr
# LangChain 의 ChatOpenAI 를 불러옵니다.
from langchain_openai import ChatOpenAI

# ChatOpenAI 인스턴스 생성
#llm = ChatOpenAI(model="gpt-3.5-turbo")
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

# 채팅봇의 응답을 처리하는 함수를 정의합니다.
def chat_respond(message, chat_history):  
    response = llm.invoke([{"role": "user", "content": message}])
    bot_message = response.content
    
    chat_history.append({"role": "user", "content": message})
    chat_history.append({"role": "assistant", "content": bot_message})

    # 수정된 채팅 기록을 반환합니다.
    return "", chat_history  

# gr.Blocks()를 사용하여 인터페이스를 생성합니다.
with gr.Blocks() as demo:  
    chatbot = gr.Chatbot(label="채팅창", type="messages")  # '채팅창'이라는 레이블을 가진 채팅봇 컴포넌트를 생성합니다.
    msg = gr.Textbox(label="입력")  # '입력'이라는 레이블을 가진 텍스트박스를 생성합니다.
    clear = gr.Button("초기화")  # '초기화'라는 레이블을 가진 버튼을 생성합니다.

    # 텍스트박스에 메시지를 입력하고 제출하면 respond 함수가 호출되도록 합니다.
    msg.submit(chat_respond, [msg, chatbot], [msg, chatbot])  
    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화합니다.
    clear.click(lambda: None, None, chatbot, queue=False)  

# 인터페이스를 실행합니다. 실행하면 사용자는 '입력' 텍스트박스에 메시지를 작성하고 제출할 수 있으며, '초기화' 버튼을 통해 채팅 기록을 초기화 할 수 있습니다.
demo.launch(debug=True)  